# Coined Model 

In [3]:
import math
import numpy as np
import scipy as sp
import sympy as sy
import networkx
from numpy import linalg as LA
from random import random,randint
import cmath 
from neblina import *
import scipy.special
import time
import sys

from CoinedModel import *

float_ = 2
complex_ = 13
init_engine(0)

In [4]:
G = networkx.complete_graph(4)

In [5]:
A = networkx.linalg.graphmatrix.adjacency_matrix(G) # Sparse matrix
#n = A.shape[0]
A = A.todense()
#A

In [6]:
S = ShiftOperator(A)

/usr/lib64/python3.6/site-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [7]:
#S.todense()

In [8]:
C = CoinOperator(A)
#C.todense()

In [9]:
psi0 = UniformInitialCondition(A)
Uprime = EvolutionOperator_SearchCoinedModel(A)
#Uprime

In [10]:
def vec_f_(V):
#    printpsi0("test_vector_sparse_matrix_multiplication_complex")
    n = V.shape[0]
    vec_f = vector_new(n, complex_)
    for i in range(n):
        vector_set(vec_f, i, V[i], 0)
    return vec_f
        
def smat_f_(M):
#    print("test_vector_sparse_matrix_multiplication_complex")
    n = M.shape[0]

    smat_f = sparse_matrix_new(n, n, complex_)
    
    for i in range(0,n):
        for j in range(0,n):
            if M[i,j] != 0.:
                sparse_matrix_set(smat_f, i, j, M[i,j], 0)

#    sparse_matrix_pack(smat_f)
    return smat_f

start = time.time()
vec_psi0 = vec_f_(psi0)
#smat_S = smat_f_(S)
#smat_C = smat_f_(C)
smat_Uprime = smat_f_(Uprime)
print(time.time() - start)

0.0004801750183105469


In [11]:
def test_vector_sparse_matrix_multiplication_complex_S_C(vec_f,smat_f,smat_f2,N):
#    print("test_vector_sparse_matrix_multiplication_complex")
    print(1)
    
    sparse_matrix_pack(smat_f)
    sparse_matrix_pack(smat_f2)
    
    move_vector_device(vec_f)
    move_sparse_matrix_device(smat_f)
    move_sparse_matrix_device(smat_f2)

    res = sparse_matvec_mul(vec_f, smat_f)
    res = sparse_matvec_mul(res, smat_f2)
    
    for i in range(N-1):
        res = sparse_matvec_mul(res, smat_f)
        res = sparse_matvec_mul(res, smat_f2)
    
    print(5)

    out = move_vector_host(res)
    MV = np.matrix([vector_get(out, 2 * i) + vector_get(out, 2 * i + 1)*complex(0,1) for i in range(n) ]).T
    
    
    return MV

In [12]:
def test_vector_sparse_matrix_multiplication_complex_U(vec_f,smat_f,dim_U,N):
#    print("test_vector_sparse_matrix_multiplication_complex")
    print(1)
    
    sparse_matrix_pack(smat_f)
    
    move_vector_device(vec_f)
    move_sparse_matrix_device(smat_f)

    res = sparse_matvec_mul(vec_f, smat_f)
    
    for i in range(N-1):
        res = sparse_matvec_mul(res, smat_f)
    
    print(2)

    out = move_vector_host(res)
    MV = np.matrix([vector_get(out,2*i)+vector_get(out,2*i+1)*complex(0,1) for i in range(n)]).T
        
    return MV

In [17]:
N = 10
n = Uprime.shape[0]
start = time.time()
parallel_Result = test_vector_sparse_matrix_multiplication_complex_U(vec_psi0,smat_Uprime,n,N)
print(time.time() - start)

1
2
0.0006461143493652344


In [14]:
n = Uprime.shape[0]
serial_Result = np.zeros((n,n))
start = time.time()
serial_Result = (Uprime**N)*psi0
print(time.time() - start)

0.0001316070556640625


In [15]:
print(LA.norm(serial_Result - parallel_Result))

5.926969055564841e-16


In [20]:
stop_engine()